In [1]:
%cd /Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function


In [2]:
import pandas as pd
import M2F

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/M2F/dependencies/go-basic.obo: fmt(1.2) rel(2025-07-22) 43,230 Terms


In [3]:
# ENV
from os import getenv
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
M2F.configure_logging(getenv("LOGS_DIR"))

# Load and clean the data

In [28]:
df = pd.read_csv(getenv("FETCHED_DATA"))

In [29]:
df.drop(columns=["Protein families", "Rhea ID"], inplace=True) # these are redundant columns -- we won't use them in training

In [30]:
clean_df = M2F.clean_cols(df,
                          col_names=["Domain [FT]",
                                     "Domain [CC]",
                                     "Gene Ontology (molecular function)",
                                     "Gene Ontology (biological process)",
                                     "Function [CC]",
                                     "Catalytic activity",
                                     "EC number",
                                     "Pathway",
                                     "Cofactor",
                                    "Sequence"],
                          apply_norms={"Domain [FT]" : False,
                                    "Domain [CC]" : True,
                                    "Gene Ontology (molecular function)" : False,
                                    "Gene Ontology (biological process)" : False,
                                    "Function [CC]" : True,
                                    "Catalytic activity" : False,
                                    "EC number" : False,
                                    "Pathway" : True,
                                    "Cofactor" : False,
                                    "Sequence" : False})

In [31]:
clean_df_portion = clean_df.head(n=30_000).copy()

In [19]:
clean_df_portion

,Entry,Domain [FT],Domain [CC],Gene Ontology (molecular function),Gene Ontology (biological process),Function [CC],Catalytic activity,EC number,Pathway,Cofactor,Sequence
0,P77499,"(2..246,)",(),"(GO:0005524, GO:0016887)","(GO:0016226, GO:0009314)",(has low atpase activity the sufbcd complex ac...,(),(),(),(),(MLSIKDLHVSVEDKAILRGLSLDVHPGEVHAIMGPNGSGKSTLSA...
1,P0DP70,"(1..114,)",(),(),"(GO:0015716, GO:0055085)",(c-terminal fragment of the phne protein part ...,(),(),(),(),(MLALFIHTTGVLSKLLSEAVEAIEPGPVEGIRATGANKLEEILYG...
2,Q8A8S8,"(35..222,)",(),"(GO:0033819,)","(GO:0036211,)",(catalyzes the transfer of endogenously produc...,(octanoyl-[ACP] + L-lysyl-[protein] = N(6)-oct...,"(2.3.1.181,)",(protein modification protein lipoylation via ...,(),(MKTVLVDWNLIPYAEAWQRQTEWFDTLVRAKAQGEAYENRIIMCE...
3,P76278,(),(),(),"(GO:0006825,)",(),(),(),(),(),(MLAFTWIALRFIHFTSLMLVFGFAMYGAWLAPLTIRRLLAKRFLR...
4,P77378,(),(),"(GO:0009055, GO:0050660)","(GO:0033539,)","(may play a role in a redox process,)",(),(),(),(),(MSQLNSVWVFSDNPERYAELFGGAQQWGQQVYAIVQNTDQAQAVM...
5,A7ZLQ4,(),(),"(GO:0016862,)",(),(),(),"(5.3.2.-,)",(),(),(MPHIDIKCFPRELDEQQKAALAADITDVIIRHLNSKDSSISIALQ...
6,A0A2D1TVM1,(),(),"(GO:0016757, GO:0015648)","(GO:0051301, GO:0071555, GO:0009252, GO:0008360)",(peptidoglycan polymerase that is essential fo...,([GlcNAc-(1->4)-Mur2Ac(oyl-L-Ala-gamma-D-Glu-L...,"(2.4.99.28,)",(cell wall biogenesis peptidoglycan biosynthes...,(),(MRRPDSDRASSRRSTPRSGRGGQSFSERYIAGVPARIMRPRLIFM...
7,A0A0J1ZC23,"(33..280,)",(),"(GO:0008409, GO:0017108, GO:0003677, GO:000028...","(GO:0033567,)",(has flap endonuclease activity during dna rep...,(),"(3.1.-.-,)",(),"(K(+), Mg(2+))",(MRGLFPISHPAVACSGIECYPYRLIFKGVIVAVHLLIVDALNLIR...
8,W4UUZ6,"(593..667,)",(),"(GO:0003984, GO:1990610, GO:0050660, GO:000028...","(GO:0009097, GO:0009099)",(),"(2 pyruvate + H(+) = (2S)-2-acetolactate + CO2,)","(2.2.1.6,)",(amino-acid biosynthesis l-isoleucine biosynth...,"(Mg(2+), thiamine diphosphate)",(MSSENKTTATSPALPQGEAPQISGAEALMRSLEHQGVKTIFGYPG...
9,A0A174FMU0,"(9..247, 299..442)",(),"(GO:0009011, GO:0004373)","(GO:0005978,)",(synthesizes alpha-14-glucan chains using adp-...,([(1->4)-alpha-D-glucosyl](n) + ADP-alpha-D-gl...,"(2.4.1.21,)","(glycan biosynthesis glycogen biosynthesis,)",(),(MAETNKKMQIVFASAECAPFVKTGGLGDVAGSLPAALVRAGAEVI...


# Initialize embedding utils and encode all of the data

In [10]:
txt_embedder = M2F.FreeTXTEmbedder(getenv("OPENAI_API_KEY"), model="LARGE_OPENAI_MODEL",
                                   cache_file_path=getenv("DB"), caching_mode="APPEND")
aa_embedder = M2F.AAChainEmbedder()

In [12]:
M2F.embed_ft_domains(clean_df_portion, aa_embedder, inplace=True)
M2F.embed_AAsequences(clean_df_portion, aa_embedder, inplace=True)
M2F.embed_freetxt_cols(clean_df_portion, ["Domain [CC]", "Function [CC]", "Catalytic activity", "Pathway"], txt_embedder, inplace=True)
M2F.encode_go(clean_df_portion, "Gene Ontology (molecular function)", coverage_target=0.9, inplace=True)
M2F.encode_go(clean_df_portion, "Gene Ontology (biological process)", coverage_target=0.9, inplace=True)
M2F.encode_ec(clean_df_portion, "EC number", coverage_target=1.0, inplace=True)
M2F.encode_multihot(clean_df_portion, "Cofactor", inplace=True)

clean_df_portion.sort_values(by="Entry", inplace=True)
clean_df_portion.sort_index(axis=1, inplace=True)

In [32]:
M2F.encode_ec(clean_df_portion, "EC number", coverage_target=0.9, inplace=True)

(            Entry Domain [FT] Domain [CC]  \
 0          P77499   (2..246,)          ()   
 1          P0DP70   (1..114,)          ()   
 2          Q8A8S8  (35..222,)          ()   
 3          P76278          ()          ()   
 4          P77378          ()          ()   
 ...           ...         ...         ...   
 29995  A0A343JAL5   (2..100,)          ()   
 29996      X1JE95   (6..117,)          ()   
 29997  A0A015TUZ8    (1..74,)          ()   
 29998  A0A062WVM1   (6..240,)          ()   
 29999  A0A0A6ZNE3    (3..77,)          ()   
 
          Gene Ontology (molecular function)  \
 0                  (GO:0005524, GO:0016887)   
 1                                        ()   
 2                             (GO:0033819,)   
 3                                        ()   
 4                  (GO:0009055, GO:0050660)   
 ...                                     ...   
 29995                         (GO:0004029,)   
 29996  (GO:0005525, GO:0043024, GO:0019843)   
 29997        

In [33]:
clean_df_portion

,Entry,Domain [FT],Domain [CC],Gene Ontology (molecular function),Gene Ontology (biological process),Function [CC],Catalytic activity,EC number,Pathway,Cofactor,Sequence
0,P77499,"(2..246,)",(),"(GO:0005524, GO:0016887)","(GO:0016226, GO:0009314)",(has low atpase activity the sufbcd complex ac...,(),NaN,(),(),(MLSIKDLHVSVEDKAILRGLSLDVHPGEVHAIMGPNGSGKSTLSA...
1,P0DP70,"(1..114,)",(),(),"(GO:0015716, GO:0055085)",(c-terminal fragment of the phne protein part ...,(),NaN,(),(),(MLALFIHTTGVLSKLLSEAVEAIEPGPVEGIRATGANKLEEILYG...
2,Q8A8S8,"(35..222,)",(),"(GO:0033819,)","(GO:0036211,)",(catalyzes the transfer of endogenously produc...,(octanoyl-[ACP] + L-lysyl-[protein] = N(6)-oct...,"(538,)",(protein modification protein lipoylation via ...,(),(MKTVLVDWNLIPYAEAWQRQTEWFDTLVRAKAQGEAYENRIIMCE...
3,P76278,(),(),(),"(GO:0006825,)",(),(),NaN,(),(),(MLAFTWIALRFIHFTSLMLVFGFAMYGAWLAPLTIRRLLAKRFLR...
4,P77378,(),(),"(GO:0009055, GO:0050660)","(GO:0033539,)","(may play a role in a redox process,)",(),NaN,(),(),(MSQLNSVWVFSDNPERYAELFGGAQQWGQQVYAIVQNTDQAQAVM...
...,...,...,...,...,...,...,...,...,...,...,...
29995,A0A343JAL5,"(2..100,)",(),"(GO:0004029,)","(GO:0006081,)",(),(),NaN,(),(),(MIKPSEHTVNVSLLLEKLINNTFDSKYIRVINPLGGKETIEELLK...
29996,X1JE95,"(6..117,)",(),"(GO:0005525, GO:0043024, GO:0019843)","(GO:0000028,)",(),(),NaN,(),(),(MKNNFKAGFVTVIGKPNVGKSTLINQLIGEKIAIVSPRPQTTRNI...
29997,A0A015TUZ8,"(1..74,)",(),(),(),(),(),NaN,(),(),(MLDSDLSALYGVETRRLNEQVKRNMERFPDDFMFQLSKDELDNLM...
29998,A0A062WVM1,"(6..240,)",(),"(GO:0015424, GO:0005524, GO:0016887)",(),(),(),NaN,(),(),(MAKLKIDVNDLHKHYGKNEVLKGITTKFYEGDVVCIIGPSGSGKS...


In [ ]:
M2F.encode_ec(clean_df_portion, "EC number", coverage_target=1.0, inplace=True)

In [38]:
len(clean_df_portion["EC number"][clean_df_portion.loc[:, "EC number"].notna()])

18236

In [14]:
clean_df_portion.loc[:, "EC number"]

7      (9,)
23    (11,)
20    (10,)
14    (18,)
21      NaN
9      (5,)
10    (13,)
15    (14,)
16     (7,)
6      (6,)
22     (2,)
17     (1,)
24     (8,)
11    (12,)
18    (16,)
5     (17,)
12    (15,)
19     (0,)
1       NaN
3       NaN
4       NaN
0       NaN
2      (4,)
13      NaN
8      (3,)
Name: EC number, dtype: object

# Save the df and then load to see the clean and numerically encoded data

In [12]:
M2F.save_df(clean_df_portion, name="example")

In [13]:
M2F.load_df("/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/example.zip")

,Catalytic activity,Cofactor,Domain [CC],Domain [FT],EC number,Entry,Function [CC],Gene Ontology (biological process),Gene Ontology (molecular function),Pathway,Sequence
0,NaN,"(1, 2)",NaN,"[0.13379566, -0.060112715, 0.49114436, 0.06966...","(9,)",A0A0J1ZC23,"[-0.05100752, -0.00023786846, -0.011066927, -0...","(1,)","(1, 5, 6)",NaN,"[0.10255164, -0.10563319, 0.38230208, 0.127237..."
1,"[-0.015551681, -0.0021475311, -0.010218693, 0....",NaN,NaN,"[0.40896338, -0.035719194, -0.1859013, -0.2392...","(11,)",A0A0N7J7Y2,NaN,"(1,)","(1,)",NaN,"[0.2541889, -0.339463, 0.113045506, -0.2062210..."
2,NaN,"(0,)",NaN,"[0.36987135, -0.13736856, -0.16509527, -0.2448...","(10,)",A0A0P0GHV0,NaN,"(1,)","(1,)",NaN,"[0.2930923, -0.4519677, -0.12289334, 0.1196395..."
3,"[-0.010819013, -0.00091222016, 0.0063029095, 0...",NaN,NaN,"[0.057893768, -0.26055378, -0.5186857, -0.2708...","(18,)",A0A173XL87,NaN,"(1,)","(1, 4, 7)",NaN,"[0.10617826, -0.22706063, -0.49131176, -0.0819..."
4,NaN,NaN,NaN,NaN,NaN,A0A173YBD4,NaN,"(1, 11)","(1,)",NaN,"[0.5116541, -0.27717066, -0.15654404, 0.004170..."
5,"[0.0012034841, 0.038362224, -0.005351928, 0.00...",NaN,NaN,"[-1.0119891, -0.85459846, -0.8319432, 0.615693...","(5,)",A0A174FMU0,"[0.017286675, 0.028886942, 0.001251799, -0.001...","(1,)","(2, 3)","[-0.009604247, 0.056476507, -0.011268787, 0.00...","[0.152202, -0.07339708, -0.23733713, 0.5565645..."
6,"[-0.018536428, -0.0124734165, -0.0040081423, 0...",NaN,NaN,"[0.6780791, -0.62408143, -0.74252397, 0.113041...","(13,)",A0A174G8M5,NaN,"(1,)","(1,)",NaN,"[0.5707834, -0.6171031, -0.6198442, 0.01490820..."
7,"[-0.010819013, -0.00091222016, 0.0063029095, 0...",NaN,NaN,"[0.14752838, -0.19741392, -0.13672641, -0.2462...","(14,)",A0A176U1F6,"[-0.016581077, 0.014960919, 0.004221898, -0.03...","(1,)","(4, 7)",NaN,"[0.22552899, -0.06948886, -0.18292694, -0.0361..."
8,"[-0.0065136887, 0.015515753, 0.0030802398, 0.0...",NaN,NaN,"[-0.020345965, -0.21926206, -0.50934, 0.237177...","(7,)",A0A1I0N6K0,NaN,NaN,"(0,)",NaN,"[-0.024644883, -0.54752254, -0.4563743, 0.0705..."
9,"[0.0043254523, 0.018771721, -0.006489864, 0.00...",NaN,NaN,NaN,"(6,)",A0A2D1TVM1,"[-0.02175193, 0.00042575714, -0.0032149185, 0....","(1,)","(1,)","[-0.015242205, 0.031689215, -0.0056352727, 0.0...","[0.54539245, -0.032479122, -0.1317679, 0.93089..."
